In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sat Nov 29 15:25:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Dataset 09**

In [ ]:
!unzip  /content/Segmentation.zip -d dataset09

Streaming output truncated to the last 5000 lines.
  inflating: dataset09/train/labels/tooth_discolation-165-_jpeg.rf.06d9c109da2f43d1bbbb2fc846d4935b.txt  
  inflating: dataset09/train/labels/tooth_discolation-1651-_jpeg.rf.90c0ad8108b8b31180cb2e52ede5bfd3.txt  
  inflating: dataset09/train/labels/tooth_discolation-1652-_jpeg.rf.c2342a83876d4916b3acf3c2e6f52afe.txt  
  inflating: dataset09/train/labels/tooth_discolation-1654-_jpeg.rf.f73217ee05153f5b215b3936049d088c.txt  
  inflating: dataset09/train/labels/tooth_discolation-1655-_jpeg.rf.6714c49b43889e9c52dd04efa9f3d452.txt  
  inflating: dataset09/train/labels/tooth_discolation-1656-_jpeg.rf.cbd1df250c4acd929c73d3e6ec802d80.txt  
  inflating: dataset09/train/labels/tooth_discolation-1657-_jpeg.rf.6bf8f03ead842be3c7a71759a109fdb6.txt  
  inflating: dataset09/train/labels/tooth_discolation-166-_jpeg.rf.b2e768b243a06aaa74f95f7b9d2271ca.txt  
  inflating: dataset09/train/labels/tooth_discolation-1660-_jpeg.rf.8b75a1f25b48fa24330dd76382d

In [1]:
import os
import yaml
from IPython.display import display, Markdown

# --- STEP 1: DEFINE PATHS AND MAPPING ---

# **IMPORTANT: CHANGE THIS LINE TO YOUR ACTUAL DATASET ROOT PATH**
DATASET_ROOT = '/content/drive/MyDrive/IndividualRelabledDatasets/dataset_09'

# The original class indices for this dataset (Segmentation_DENTAL_AI) are:
# 0: 'Caries'
# 1: 'Calculus'
# 2: 'Gingivitis'
# 3: 'Hypodontia'
# 4: 'Tooth discoloration'
# 5: 'Ulcer'

# Map OLD class index to NEW class index
# New classes: ['Cavity', 'Non-Cavity'] (Indices 0, 1)
CLASS_MAPPING = {
    # Decay/Cavity class -> New Index 0 ('Cavity')
    0: 0, # 'Caries'

    # Non-Cavity/Other Disease classes -> New Index 1 ('Non-Cavity')
    1: 1, # 'Calculus'
    2: 1, # 'Gingivitis'
    3: 1, # 'Hypodontia'
    4: 1, # 'Tooth discoloration'
    5: 1, # 'Ulcer'
}

# Directories to process (assuming the standard Roboflow structure)
LABEL_DIRS = ['train/labels', 'valid/labels', 'test/labels']

display(Markdown(f"## ⚙️ Starting Class Re-Mapping"))
display(Markdown(f"**Dataset Root Directory:** `{DATASET_ROOT}`"))
display(Markdown(f"**Mapping:** {CLASS_MAPPING}"))
display(Markdown("---"))


# --- STEP 2: DEFINE THE LABEL UPDATE FUNCTION (No Change) ---

def update_yolo_labels(labels_dir_relative, mapping):
    """Updates class indices in YOLO .txt label files based on the mapping."""
    full_path = os.path.join(DATASET_ROOT, labels_dir_relative)

    if not os.path.exists(full_path):
        print(f"⚠️ Warning: Directory not found: {full_path}. Skipping.")
        return

    processed_count = 0
    for filename in os.listdir(full_path):
        if not filename.endswith('.txt'):
            continue

        filepath = os.path.join(full_path, filename)
        new_lines = []

        with open(filepath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            try:
                old_index = int(parts[0])
            except ValueError:
                continue

            # Check if the old index is in the mapping
            if old_index in mapping and mapping[old_index] != -1:
                new_index = mapping[old_index]
                # Reconstruct the line with the new index
                new_line = f"{new_index} {' '.join(parts[1:])}\n"
                new_lines.append(new_line)

        # Write the updated lines back to the file
        with open(filepath, 'w') as f:
            f.writelines(new_lines)
            processed_count += 1

    print(f"✅ Successfully processed {processed_count} label files in {labels_dir_relative}")


# --- STEP 3: EXECUTE LABEL UPDATES (No Change) ---

for label_dir in LABEL_DIRS:
    update_yolo_labels(label_dir, CLASS_MAPPING)


# --- STEP 4: UPDATE THE data.yaml FILE ---

yaml_path = os.path.join(DATASET_ROOT, 'data.yaml')

display(Markdown("---"))
display(Markdown("## 📄 Updating `data.yaml`"))

# Load the existing data.yaml
if not os.path.exists(yaml_path):
    print(f"❌ Error: data.yaml not found at {yaml_path}. Cannot proceed with final step.")
else:
    with open(yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)

    # Update the number of classes (nc) to 2
    data_yaml['nc'] = 2

    # Update the class names to the new binary labels
    data_yaml['names'] = ['Cavity', 'Not Cavity']

    # Remove the roboflow section to clean up the config
    if 'roboflow' in data_yaml:
        del data_yaml['roboflow']

    # Write the updated data.yaml back to the file
    with open(yaml_path, 'w') as f:
        yaml.safe_dump(data_yaml, f, sort_keys=False) # sort_keys=False to keep a clean look

    print(f"✅ Successfully updated {yaml_path}")
    display(Markdown("### New `data.yaml` Contents:"))
    !cat {yaml_path}

## ⚙️ Starting Class Re-Mapping

**Dataset Root Directory:** `/content/drive/MyDrive/IndividualRelabledDatasets/dataset_09`

**Mapping:** {0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}

---

✅ Successfully processed 0 label files in train/labels
✅ Successfully processed 0 label files in valid/labels
✅ Successfully processed 0 label files in test/labels


---

## 📄 Updating `data.yaml`

✅ Successfully updated /content/drive/MyDrive/IndividualRelabledDatasets/dataset_09/data.yaml


### New `data.yaml` Contents:

train: train/images
val: valid/images
test: test/images
names:
- Cavity
- Not Cavity
nc: 2


In [ ]:
# The '-r' flag stands for recursive (includes all subdirectories).
# The output zip file will be named 'my_yolo_dataset.zip'
!zip -r /content/my_yolo_dataset09.zip /content/dataset09

Streaming output truncated to the last 5000 lines.
  adding: content/dataset09/train/images/caries-644-_jpeg.rf.d9c4a7420105ca756088ad93b01ced4f.jpg (deflated 3%)
  adding: content/dataset09/train/images/calcu-350-copy_jpg.rf.531fd2b34370e2949e47ff3bbc570f28.jpg (deflated 1%)
  adding: content/dataset09/train/images/caries-1976-_jpeg.rf.a74872d10b38a2056375a49694ff485b.jpg (deflated 1%)
  adding: content/dataset09/train/images/hypodontia-668-_jpg.rf.f1a37a6629efb62e513a3eacb1dfb1d2.jpg (deflated 1%)
  adding: content/dataset09/train/images/caries-1260-_jpeg.rf.3e7930076addbd3c8f58f2b48745f1e8.jpg (deflated 3%)
  adding: content/dataset09/train/images/tooth_discolation-340-_jpeg.rf.a304db1760ac222f89e85383cbaa1f27.jpg (deflated 1%)
  adding: content/dataset09/train/images/gingivitis-1250-_jpg.rf.6f0a65228e282f88d17aef6ab4a541e9.jpg (deflated 0%)
  adding: content/dataset09/train/images/hypodontia-250-_jpg.rf.117fccd068ba8dbe866b8b534db05769.jpg (deflated 2%)
  adding: content/dataset09/

In [ ]:
import os
import yaml
from IPython.display import display, Markdown

# --- STEP 1: DEFINE PATHS AND MAPPING ---

# **IMPORTANT: CHANGE THIS LINE TO YOUR ACTUAL DATASET ROOT PATH**
DATASET_ROOT = '/content/dataset04'

# The original class indices for this dataset are:
# 0: 'Caries'
# 1: 'Cavity'
# 2: 'Crack'
# 3: 'Tooth'

# Map OLD class index to NEW class index
# New classes: ['Cavity', 'Non-Cavity'] (Indices 0, 1)
CLASS_MAPPING = {
    # Decay/Cavity classes -> New Index 0 ('Cavity')
    0: 0, # 'Caries'
    1: 0, # 'Cavity'

    # Non-Cavity/Structural classes -> New Index 1 ('Non-Cavity')
    2: 1, # 'Crack'
    3: 1, # 'Tooth'
}

# Directories to process (relative paths based on the structure provided in your data.yaml)
# Note: The paths provided in your YAML are relative to the root: train/images, val/images, etc.
# So, the labels will be in the corresponding labels directories.
LABEL_DIRS = ['train/labels', 'valid/labels', 'test/labels']

display(Markdown(f"## ⚙️ Starting Class Re-Mapping"))
display(Markdown(f"**Dataset Root Directory:** `{DATASET_ROOT}`"))
display(Markdown(f"**Mapping:** {CLASS_MAPPING}"))
display(Markdown("---"))


# --- STEP 2: DEFINE THE LABEL UPDATE FUNCTION (No Change) ---

def update_yolo_labels(labels_dir_relative, mapping):
    """Updates class indices in YOLO .txt label files based on the mapping."""
    full_path = os.path.join(DATASET_ROOT, labels_dir_relative)

    if not os.path.exists(full_path):
        print(f"⚠️ Warning: Directory not found: {full_path}. Skipping.")
        return

    processed_count = 0
    for filename in os.listdir(full_path):
        if not filename.endswith('.txt'):
            continue

        filepath = os.path.join(full_path, filename)
        new_lines = []

        with open(filepath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue

            try:
                old_index = int(parts[0])
            except ValueError:
                continue

            # Check if the old index is in the mapping
            if old_index in mapping and mapping[old_index] != -1:
                new_index = mapping[old_index]
                # Reconstruct the line with the new index
                new_line = f"{new_index} {' '.join(parts[1:])}\n"
                new_lines.append(new_line)

        # Write the updated lines back to the file
        with open(filepath, 'w') as f:
            f.writelines(new_lines)
            processed_count += 1

    print(f"✅ Successfully processed {processed_count} label files in {labels_dir_relative}")


# --- STEP 3: EXECUTE LABEL UPDATES (No Change) ---

for label_dir in LABEL_DIRS:
    update_yolo_labels(label_dir, CLASS_MAPPING)


# --- STEP 4: UPDATE THE data.yaml FILE ---

yaml_path = os.path.join(DATASET_ROOT, 'data.yaml')

display(Markdown("---"))
display(Markdown("## 📄 Updating `data.yaml`"))

# Load the existing data.yaml
if not os.path.exists(yaml_path):
    print(f"❌ Error: data.yaml not found at {yaml_path}. Cannot proceed with final step.")
else:
    with open(yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)

    # Update the number of classes (nc) to 2
    data_yaml['nc'] = 2

    # Update the class names to the new binary labels
    data_yaml['names'] = ['Cavity', 'Non-Cavity']

    # Remove the roboflow section to clean up the config
    if 'roboflow' in data_yaml:
        del data_yaml['roboflow']

    # Write the updated data.yaml back to the file
    with open(yaml_path, 'w') as f:
        yaml.safe_dump(data_yaml, f, sort_keys=False) # sort_keys=False to keep a clean look

    print(f"✅ Successfully updated {yaml_path}")
    display(Markdown("### New `data.yaml` Contents:"))
    !cat {yaml_path}

## ⚙️ Starting Class Re-Mapping

**Dataset Root Directory:** `/content/dataset04`

**Mapping:** {0: 0, 1: 0, 2: 1, 3: 1}

---

✅ Successfully processed 5375 label files in train/labels
✅ Successfully processed 254 label files in valid/labels
✅ Successfully processed 446 label files in test/labels


---

## 📄 Updating `data.yaml`

✅ Successfully updated /content/dataset04/data.yaml


### New `data.yaml` Contents:

train: train/images
val: valid/images
test: test/images
names:
- Cavity
- Non-Cavity
nc: 2


In [ ]:
!zip -r /content/my_yolo_dataset04.zip /content/dataset04

Streaming output truncated to the last 5000 lines.
  adding: content/dataset04/train/images/1821_jpg.rf.aea984ffc1d2c8ba135ad8c9085abafc.jpg (deflated 1%)
  adding: content/dataset04/train/images/2602_jpg.rf.f4f930e7b3e82022dfe4dba13eebeb79.jpg (deflated 2%)
  adding: content/dataset04/train/images/298_jpg.rf.bd1bbed1c39a195bd3849d1fcb247026.jpg (deflated 1%)
  adding: content/dataset04/train/images/2750_jpg.rf.1a52fe20d38f7b695fcf7d66a8b12941.jpg (deflated 1%)
  adding: content/dataset04/train/images/294_jpg.rf.cd79e6f5568eeaebb6d480b6a0e0a134.jpg (deflated 3%)
  adding: content/dataset04/train/images/1075_jpg.rf.b00b6bec93be59f14ef57679f9be5958.jpg (deflated 1%)
  adding: content/dataset04/train/images/2598_jpg.rf.e4ae74122422311565b7c7498bf8d29b.jpg (deflated 1%)
  adding: content/dataset04/train/images/1069_jpg.rf.d38cded86691c2640a9088a12f9b86d6.jpg (deflated 2%)
  adding: content/dataset04/train/images/163_jpg.rf.dc246c86e7d8eae16fc39c1847e92d2b.jpg (deflated 0%)
  adding: conten

In [ ]:
from google.colab import files

files.download('/content/my_yolo_dataset04.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Testing Code for Making the +ve and -ve classes separate:

In [ ]:
!unzip /content/newDataset/dataset.zip -d newTestDataset

Archive:  /content/newDataset/dataset.zip
   creating: newTestDataset/dataset/Images/
   creating: newTestDataset/dataset/Labels/
  inflating: newTestDataset/dataset/Images/anonymous_003-007-1130-00_1732522239159_Frontal_View.jpg  
  inflating: newTestDataset/dataset/Images/anonymous_003-007-1208-00_1732856242544_Frontal_View.jpg  
  inflating: newTestDataset/dataset/Images/anonymous_003-007-1212-00_1732860958536_Frontal_View.jpg  
  inflating: newTestDataset/dataset/Images/anonymous_003-007-1214-00_1732862917124_Frontal_View.jpg  
  inflating: newTestDataset/dataset/Images/anonymous_003-007-1361-00_1733724284664_Frontal_View.jpg  
  inflating: newTestDataset/dataset/Images/anonymous_003-007-1477-00_1734160734308_Frontal_View.jpg  
  inflating: newTestDataset/dataset/Images/anonymous_003-007-607-00_1729059680698_Frontal_View.jpg  
  inflating: newTestDataset/dataset/Images/anonymous_003-007-673-00_1729239709286_Frontal_View.jpg  
  inflating: newTestDataset/dataset/Images/anonymous_003

In [ ]:
import os
import shutil
from tqdm import tqdm
from IPython.display import display, Markdown

# --- CONFIGURATION ---

# ⚠️ UPDATE THE BASE DIRECTORY WHERE YOUR UNZIPPED DATA IS LOCATED
# Assuming the structure is: /content/dataset/Images and /content/dataset/Labels
BASE_DIR = '/content/newDataset/dataset'

# Define the names of the source folders inside BASE_DIR
SOURCE_IMAGES_DIR = os.path.join(BASE_DIR, 'Images')
SOURCE_LABELS_DIR = os.path.join(BASE_DIR, 'Labels')


# Define the target root directories
TARGET_ROOT_DIR = '/content/Split_Data' # Root for all new data
CAVITY_DIR = os.path.join(TARGET_ROOT_DIR, 'Cavity_Data')
NON_CAVITY_DIR = os.path.join(TARGET_ROOT_DIR, 'NonCavity_Data')

# Define the class index that corresponds to 'Cavity' in your current YOLO labels.
# Based on our re-labeling steps, this should be index 0.
CAVITY_CLASS_INDEX = 0

# --- SETUP AND FOLDER CREATION ---

display(Markdown("## 📁 Setting Up Directories"))

# Create all necessary target directories
os.makedirs(os.path.join(CAVITY_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(CAVITY_DIR, 'labels'), exist_ok=True)
os.makedirs(os.path.join(NON_CAVITY_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(NON_CAVITY_DIR, 'labels'), exist_ok=True)

print(f"Target directories created under: {TARGET_ROOT_DIR}")

# --- PROCESSING FUNCTION ---

def contains_cavity_label(label_filepath, target_index):
    """Checks if the YOLO label file contains an instance of the target class index."""
    try:
        with open(label_filepath, 'r') as f:
            for line in f:
                # YOLO format: [class_index] [x_center] [y_center] [width] [height]
                if line.strip().startswith(str(target_index) + ' '):
                    return True
    except Exception as e:
        print(f"Error reading label file {label_filepath}: {e}")
        return False
    return False

# --- MAIN EXECUTION ---

display(Markdown("## 🏃 Starting Data Separation"))

cavity_count = 0
non_cavity_count = 0
skipped_count = 0

# We iterate through the label files (always .txt)
label_files = [f for f in os.listdir(SOURCE_LABELS_DIR) if f.endswith('.txt')]

# Common image extensions to check
IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png']

for label_filename in tqdm(label_files, desc="Processing files"):

    # 1. Determine base name and paths
    base_name = os.path.splitext(label_filename)[0]

    # 2. Find the corresponding image file
    image_found = False
    image_filename = None

    for ext in IMAGE_EXTENSIONS:
        temp_image_filename = base_name + ext
        if os.path.exists(os.path.join(SOURCE_IMAGES_DIR, temp_image_filename)):
            image_filename = temp_image_filename
            image_found = True
            break

    if not image_found:
        skipped_count += 1
        continue

    src_label_path = os.path.join(SOURCE_LABELS_DIR, label_filename)
    src_image_path = os.path.join(SOURCE_IMAGES_DIR, image_filename)

    # 3. Check for the 'Cavity' class and set target directories
    if contains_cavity_label(src_label_path, CAVITY_CLASS_INDEX):
        # Case A: Contains the Cavity class
        target_images_dir = os.path.join(CAVITY_DIR, 'images')
        target_labels_dir = os.path.join(CAVITY_DIR, 'labels')
        cavity_count += 1
    else:
        # Case B: Does NOT contain the Cavity class
        target_images_dir = os.path.join(NON_CAVITY_DIR, 'images')
        target_labels_dir = os.path.join(NON_CAVITY_DIR, 'labels')
        non_cavity_count += 1

    # 4. Move the files (Image and Label)
    shutil.move(src_image_path, os.path.join(target_images_dir, image_filename))
    shutil.move(src_label_path, os.path.join(target_labels_dir, label_filename))

display(Markdown("---"))
display(Markdown("## ✅ Process Complete"))
print(f"Total Files Processed (Images + Labels): {cavity_count + non_cavity_count}")
print(f"Images moved to 'Cavity_Data': {cavity_count}")
print(f"Images moved to 'NonCavity_Data': {non_cavity_count}")
if skipped_count > 0:
    print(f"⚠️ Skipped files (Image missing): {skipped_count}")
print(f"New dataset structure is in: {TARGET_ROOT_DIR}")

## 📁 Setting Up Directories

Target directories created under: /content/Split_Data


## 🏃 Starting Data Separation

Processing files: 100%|██████████| 38/38 [00:00<00:00, 8978.34it/s]


---

## ✅ Process Complete

Total Files Processed (Images + Labels): 15
Images moved to 'Cavity_Data': 9
Images moved to 'NonCavity_Data': 6
⚠️ Skipped files (Image missing): 23
New dataset structure is in: /content/Split_Data


In [ ]:
!zip -r /content/my_yolo_dataset04.zip /content/Split_Data

  adding: content/Split_Data/ (stored 0%)
  adding: content/Split_Data/NonCavity_Data/ (stored 0%)
  adding: content/Split_Data/NonCavity_Data/labels/ (stored 0%)
  adding: content/Split_Data/NonCavity_Data/labels/anonymous_003-008-1140-00_1732538036389_Frontal_View.txt (stored 0%)
  adding: content/Split_Data/NonCavity_Data/labels/anonymous_003-007-848-00_1730185308013_Frontal_View.txt (deflated 25%)
  adding: content/Split_Data/NonCavity_Data/labels/anonymous_003-007-950-00_1730792603619_Frontal_View.txt (deflated 27%)
  adding: content/Split_Data/NonCavity_Data/labels/anonymous_003-008-816-00_1729939059509_Frontal_View.txt (deflated 18%)
  adding: content/Split_Data/NonCavity_Data/labels/anonymous_003-008-625-00_1729089968039_Frontal_View.txt (stored 0%)
  adding: content/Split_Data/NonCavity_Data/labels/anonymous_003-007-673-00_1729239709286_Frontal_View.txt (deflated 31%)
  adding: content/Split_Data/NonCavity_Data/images/ (stored 0%)
  adding: content/Split_Data/NonCavity_Data/im

In [ ]:
import os
import shutil
from tqdm import tqdm
from IPython.display import display, Markdown

# --- CONFIGURATION ---

# ⚠️ BASE DIRECTORY: Adjusted based on your folder structure image: /newDataset/dataset/
BASE_DIR = '/content/newDataset/dataset'

# Define the names of the source folders inside BASE_DIR
SOURCE_IMAGES_DIR = os.path.join(BASE_DIR, 'Images')
SOURCE_LABELS_DIR = os.path.join(BASE_DIR, 'Labels')

# Define the target root directories
TARGET_ROOT_DIR = '/content/Split_Data'
CAVITY_DIR = os.path.join(TARGET_ROOT_DIR, 'Cavity_Data')
NON_CAVITY_DIR = os.path.join(TARGET_ROOT_DIR, 'NonCavity_Data')

# Define the class index that corresponds to 'Cavity' (should be 0 after re-labeling)
CAVITY_CLASS_INDEX = 0

# --- SETUP AND FOLDER CREATION ---

display(Markdown("## 📁 Setting Up Directories"))

# Create all necessary target directories
os.makedirs(os.path.join(CAVITY_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(CAVITY_DIR, 'labels'), exist_ok=True)
os.makedirs(os.path.join(NON_CAVITY_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(NON_CAVITY_DIR, 'labels'), exist_ok=True)

print(f"Target directories created under: {TARGET_ROOT_DIR}")

# --- PROCESSING FUNCTION ---

def contains_cavity_label(label_filepath, target_index):
    """Checks if the YOLO label file contains an instance of the target class index."""
    try:
        with open(label_filepath, 'r') as f:
            for line in f:
                # YOLO format: [class_index] [x_center] [y_center] [width] [height]
                if line.strip().startswith(str(target_index) + ' '):
                    return True
    except Exception as e:
        print(f"Error reading label file {label_filepath}: {e}")
        return False
    return False

# --- MAIN EXECUTION ---

display(Markdown("## 🏃 Starting Data Separation"))

cavity_count = 0
non_cavity_count = 0

try:
    # List all .txt files in the Labels directory
    label_files = [f for f in os.listdir(SOURCE_LABELS_DIR) if f.endswith('.txt')]
except FileNotFoundError:
    print(f"❌ FATAL ERROR: Source Labels directory not found. Check BASE_DIR: {SOURCE_LABELS_DIR}")
    exit()

print(f"Found {len(label_files)} label files to process.")

for label_filename in tqdm(label_files, desc="Processing files"):

    # 1. Determine base name and paths
    base_name = os.path.splitext(label_filename)[0]

    # 2. Set file names based on user's confirmation (.jpg for image, .txt for label)
    image_filename = base_name + '.jpg'

    src_label_path = os.path.join(SOURCE_LABELS_DIR, label_filename)
    src_image_path = os.path.join(SOURCE_IMAGES_DIR, image_filename)

    # Check if the image exists (this is the new point of failure if it's not .jpg)
    if not os.path.exists(src_image_path):
        print(f"\n⚠️ WARNING: Image file {src_image_path} not found. Skipping {label_filename}.")
        continue

    # 3. Check for the 'Cavity' class and set target directories
    if contains_cavity_label(src_label_path, CAVITY_CLASS_INDEX):
        # Case A: Contains the Cavity class
        target_images_dir = os.path.join(CAVITY_DIR, 'images')
        target_labels_dir = os.path.join(CAVITY_DIR, 'labels')
        cavity_count += 1
    else:
        # Case B: Does NOT contain the Cavity class (Non-Cavity or healthy)
        target_images_dir = os.path.join(NON_CAVITY_DIR, 'images')
        target_labels_dir = os.path.join(NON_CAVITY_DIR, 'labels')
        non_cavity_count += 1

    # 4. Move the files (Image and Label)
    shutil.move(src_image_path, os.path.join(target_images_dir, image_filename))
    shutil.move(src_label_path, os.path.join(target_labels_dir, label_filename))

display(Markdown("---"))
display(Markdown("## ✅ Process Complete"))
total_processed = cavity_count + non_cavity_count
total_labels_found = len(label_files)
skipped_count = total_labels_found - total_processed

print(f"Total Labels Found: {total_labels_found}")
print(f"Total Files Processed (Images + Labels): {total_processed}")
print(f"Images moved to 'Cavity_Data': {cavity_count}")
print(f"Images moved to 'NonCavity_Data': {non_cavity_count}")
if skipped_count > 0:
    print(f"⚠️ Skipped files (Image missing or not .jpg): {skipped_count}")
print(f"New dataset structure is in: {TARGET_ROOT_DIR}")

## 📁 Setting Up Directories

Target directories created under: /content/Split_Data


## 🏃 Starting Data Separation

Found 23 label files to process.


Processing files: 100%|██████████| 23/23 [00:00<00:00, 37964.97it/s]


⚠️ WARNING: Image file /content/newDataset/dataset/Images/anonymous_003-008-620-01_1729083811620_Frontal_View.jpg not found. Skipping anonymous_003-008-620-01_1729083811620_Frontal_View.txt.

⚠️ WARNING: Image file /content/newDataset/dataset/Images/anonymous_003-008-1139-01_1732537527555_Frontal_View.jpg not found. Skipping anonymous_003-008-1139-01_1732537527555_Frontal_View.txt.

⚠️ WARNING: Image file /content/newDataset/dataset/Images/anonymous_003-008-562-01_1728904090342_Frontal_View.jpg not found. Skipping anonymous_003-008-562-01_1728904090342_Frontal_View.txt.

⚠️ WARNING: Image file /content/newDataset/dataset/Images/anonymous_003-007-847-01_1730184498927_Frontal_View.jpg not found. Skipping anonymous_003-007-847-01_1730184498927_Frontal_View.txt.

⚠️ WARNING: Image file /content/newDataset/dataset/Images/anonymous_003-007-608-01_1729061556946_Frontal_View.jpg not found. Skipping anonymous_003-007-608-01_1729061556946_Frontal_View.txt.

⚠️ WARNING: Image file /content/newDa

---

## ✅ Process Complete

Total Labels Found: 23
Total Files Processed (Images + Labels): 0
Images moved to 'Cavity_Data': 0
Images moved to 'NonCavity_Data': 0
⚠️ Skipped files (Image missing or not .jpg): 23
New dataset structure is in: /content/Split_Data


In [ ]:
import os
import shutil
import glob

# --- Configuration ---
# Source directory containing images (.jpg) and labels (.txt)
SOURCE_DIR = '/content/newDataset/dataset'

# Base destination directory
BASE_DEST_DIR = '/content/Split_Data'

# Target class index to identify 'Cavity'
# Based on your request, CAVITY_CLASS_INDEX is 0.
CAVITY_CLASS_INDEX = '0'

# --- 1. Define and Create Destination Folders ---

# Define the full paths for the destination folders
DESTINATIONS = {
    'Cavity': {
        'images': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'labels')
    },
    'NonCavity': {
        'images': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'labels')
    }
}

print(f"Creating necessary directories under {BASE_DEST_DIR}...")

# Create all necessary destination directories
for category in DESTINATIONS.values():
    os.makedirs(category['images'], exist_ok=True)
    os.makedirs(category['labels'], exist_ok=True)
    print(f"Created: {category['images']}")
    print(f"Created: {category['labels']}")

# --- 2. Process and Move Files ---

# Get a list of all label files (.txt) in the source directory
label_files = glob.glob(os.path.join(SOURCE_DIR, '*.txt'))
print(f"\nFound {len(label_files)} label files to process.")

cavity_count = 0
non_cavity_count = 0

for label_path in label_files:
    # 1. Determine the corresponding image file path
    base_name = os.path.splitext(os.path.basename(label_path))[0]
    image_path = os.path.join(SOURCE_DIR, base_name + '.jpg')

    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Warning: Skipping {label_path}, corresponding image {image_path} not found.")
        continue

    is_cavity_data = True

    # 2. Read the label file content
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # 3. Check for the class index (0)
        # We assume the YOLO format: [class_index] [x_center] [y_center] [width] [height]
        for line in lines:
            if line.strip(): # Check if line is not empty
                # The first element before the first space is the class index
                class_index = line.split()[0]

                # If any line's class index is NOT 0, the entire file is NonCavity
                if class_index != CAVITY_CLASS_INDEX:
                    is_cavity_data = False
                    break # Stop checking lines for this file

    except Exception as e:
        print(f"Error reading or processing {label_path}: {e}")
        is_cavity_data = False # Treat files with errors as NonCavity or skip (NonCavity is safer)

    # 4. Move the files based on the check result

    if is_cavity_data:
        # Destination is the 'Cavity' folder
        dest_set = DESTINATIONS['Cavity']
        cavity_count += 1
    else:
        # Destination is the 'NonCavity' folder
        dest_set = DESTINATIONS['NonCavity']
        non_cavity_count += 1

    # Copy the image and label files
    try:
        shutil.copy(image_path, dest_set['images'])
        shutil.copy(label_path, dest_set['labels'])
    except Exception as e:
        print(f"Error moving files for {base_name}: {e}")

# --- 3. Final Summary ---
print("\n--- Summary ---")
print(f"✅ Processing Complete.")
print(f"Moved **{cavity_count}** files to the Cavity folders.")
print(f"Moved **{non_cavity_count}** files to the NonCavity folders.")
print(f"Total files processed: {cavity_count + non_cavity_count}")

Creating necessary directories under /content/Split_Data...
Created: /content/Split_Data/Cavity_Data/images
Created: /content/Split_Data/Cavity_Data/labels
Created: /content/Split_Data/NonCavity_Data/images
Created: /content/Split_Data/NonCavity_Data/labels

Found 0 label files to process.

--- Summary ---
✅ Processing Complete.
Moved **0** files to the Cavity folders.
Moved **0** files to the NonCavity folders.
Total files processed: 0


In [ ]:
import os
import shutil
import glob

# --- Configuration ---

# Base source directory (the parent of Images and Labels)
BASE_SOURCE_DIR = '/content/newDataset/dataset'

# Source directories for images and labels based on your structure
SOURCE_IMAGE_DIR = os.path.join(BASE_SOURCE_DIR, 'Images')
SOURCE_LABEL_DIR = os.path.join(BASE_SOURCE_DIR, 'Labels')

# Base destination directory
BASE_DEST_DIR = '/content/Split_Data'

# Target class index to identify 'Cavity' (must match the string format in your .txt files)
CAVITY_CLASS_INDEX = '0'

# --- 1. Define and Create Destination Folders ---

# Define the full paths for the destination folders
DESTINATIONS = {
    'Cavity': {
        'images': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'labels')
    },
    'NonCavity': {
        'images': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'labels')
    }
}

print(f"Creating necessary directories under {BASE_DEST_DIR}...")

# Create all necessary destination directories
for category in DESTINATIONS.values():
    os.makedirs(category['images'], exist_ok=True)
    os.makedirs(category['labels'], exist_ok=True)
    print(f"Created: {category['images']}")

# --- 2. Process and Move Files ---

# Get a list of all label files (.txt) from the Labels subfolder
label_files = glob.glob(os.path.join(SOURCE_LABEL_DIR, '*.txt'))
print(f"\nFound {len(label_files)} label files to process.")

cavity_count = 0
non_cavity_count = 0

for label_path in label_files:
    # 1. Determine the corresponding image file path
    # Get the file name (e.g., 'image_001') without the path or extension
    base_name = os.path.splitext(os.path.basename(label_path))[0]
    # Construct the full path to the image in the 'Images' folder
    image_path = os.path.join(SOURCE_IMAGE_DIR, base_name + '.jpg')

    # Check if the image file exists in the Images folder
    if not os.path.exists(image_path):
        # You might need to check for .png or other extensions if your images aren't all .jpg
        image_path_png = os.path.join(SOURCE_IMAGE_DIR, base_name + '.png')
        if os.path.exists(image_path_png):
            image_path = image_path_png # Update the path if it's a PNG
        else:
            print(f"Warning: Skipping {base_name}, corresponding image (.jpg or .png) not found in {SOURCE_IMAGE_DIR}.")
            continue

    is_cavity_data = True

    # 2. Read the label file content
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # 3. Check for the class index (0)
        # We check that *all* annotations belong to class '0'
        for line in lines:
            line = line.strip()
            if line: # Check if line is not empty
                # The first element before the first space is the class index
                # Use split() to get the list of values
                parts = line.split()
                if not parts:
                    continue # Skip empty lines that passed the .strip() check

                class_index = parts[0]

                # If any line's class index is NOT 0, the entire file is NonCavity
                if class_index != CAVITY_CLASS_INDEX:
                    is_cavity_data = False
                    break # Stop checking lines for this file

    except Exception as e:
        print(f"Error reading or processing {label_path}: {e}")
        is_cavity_data = False # Treat files with errors as NonCavity

    # 4. Move the files based on the check result

    if is_cavity_data:
        # Destination is the 'Cavity' folder
        dest_set = DESTINATIONS['Cavity']
        cavity_count += 1
    else:
        # Destination is the 'NonCavity' folder
        dest_set = DESTINATIONS['NonCavity']
        non_cavity_count += 1

    # Copy the image and label files
    try:
        shutil.copy(image_path, dest_set['images'])
        shutil.copy(label_path, dest_set['labels'])
    except Exception as e:
        print(f"Error copying files for {base_name}: {e}")

# --- 3. Final Summary ---
print("\n--- Summary ---")
print(f"✅ Processing Complete.")
print(f"Moved **{cavity_count}** files to the Cavity folders.")
print(f"Moved **{non_cavity_count}** files to the NonCavity folders.")
print(f"Total files processed: {cavity_count + non_cavity_count}")

Creating necessary directories under /content/Split_Data...
Created: /content/Split_Data/Cavity_Data/images
Created: /content/Split_Data/NonCavity_Data/images

Found 23 label files to process.

--- Summary ---
✅ Processing Complete.
Moved **0** files to the Cavity folders.
Moved **0** files to the NonCavity folders.
Total files processed: 0


In [ ]:
import os
import shutil
import glob

# --- Configuration ---

# Base source directory (the parent of Images and Labels)
BASE_SOURCE_DIR = '/content/newTestDataset/dataset'

# Source directories for images and labels based on your confirmed structure
SOURCE_IMAGE_DIR = os.path.join(BASE_SOURCE_DIR, 'Images')
SOURCE_LABEL_DIR = os.path.join(BASE_SOURCE_DIR, 'Labels')

# Base destination directory
BASE_DEST_DIR = '/content/Split_Test_Data_Final'

# Target class index to identify 'Cavity' (must match the string format in your .txt files)
CAVITY_CLASS_INDEX = '0'

# --- 1. Define and Create Destination Folders ---

# Define the full paths for the destination folders
DESTINATIONS = {
    'Cavity': {
        'images': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'labels')
    },
    'NonCavity': {
        'images': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'labels')
    }
}

print(f"Creating necessary directories under {BASE_DEST_DIR}...")

# Create all necessary destination directories
for category in DESTINATIONS.values():
    os.makedirs(category['images'], exist_ok=True)
    os.makedirs(category['labels'], exist_ok=True)
    print(f"Created: {category['images']}")

# --- 2. Process and Move Files ---

# Get a list of all label files (.txt) from the Labels subfolder
label_files = glob.glob(os.path.join(SOURCE_LABEL_DIR, '*.txt'))
print(f"\nFound {len(label_files)} label files to process.")

cavity_count = 0
non_cavity_count = 0

for label_path in label_files:
    # 1. Determine the corresponding image file path
    base_name = os.path.splitext(os.path.basename(label_path))[0]

    # Attempt to find the image with .jpg or .png extension
    image_path = os.path.join(SOURCE_IMAGE_DIR, base_name + '.jpg')

    if not os.path.exists(image_path):
        image_path_png = os.path.join(SOURCE_IMAGE_DIR, base_name + '.png')
        if os.path.exists(image_path_png):
            image_path = image_path_png # Use PNG path


    is_cavity_data = True

    # 2. Read the label file content
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # 3. Check for the class index (0)
        # We verify that *only* class '0' appears in the annotations
        for line in lines:
            line = line.strip()
            if line:
                parts = line.split()
                if not parts:
                    continue

                class_index = parts[0]

                # If any line's class index is NOT 0, it's NonCavity
                if class_index != CAVITY_CLASS_INDEX:
                    is_cavity_data = False
                    break # Optimization: stop checking lines for this file

    except Exception as e:
        print(f"Error reading or processing {label_path}: {e}")
        is_cavity_data = False # Treat files with read errors as NonCavity

    # 4. Move (Copy) the files

    if is_cavity_data:
        # Move to Cavity folders
        dest_set = DESTINATIONS['Cavity']
        cavity_count += 1
    else:
        # Move to NonCavity folders
        dest_set = DESTINATIONS['NonCavity']
        non_cavity_count += 1

    # Copy the files
    try:
        shutil.copy(image_path, dest_set['images'])
        shutil.copy(label_path, dest_set['labels'])
    except Exception as e:
        print(f"Error copying files for {base_name}: {e}")

# --- 3. Final Summary ---
print("\n--- Summary ---")
print(f"✅ Processing Complete.")
print(f"Moved **{cavity_count}** files to the Cavity folders.")
print(f"Moved **{non_cavity_count}** files to the NonCavity folders.")
print(f"Total files processed: {cavity_count + non_cavity_count}")

Creating necessary directories under /content/Split_Test_Data_Final...
Created: /content/Split_Test_Data_Final/Cavity_Data/images
Created: /content/Split_Test_Data_Final/NonCavity_Data/images

Found 38 label files to process.
Error copying files for anonymous_003-008-620-01_1729083811620_Frontal_View: [Errno 2] No such file or directory: '/content/newTestDataset/dataset/Images/anonymous_003-008-620-01_1729083811620_Frontal_View.jpg'
Error copying files for anonymous_003-008-1139-01_1732537527555_Frontal_View: [Errno 2] No such file or directory: '/content/newTestDataset/dataset/Images/anonymous_003-008-1139-01_1732537527555_Frontal_View.jpg'
Error copying files for anonymous_003-008-562-01_1728904090342_Frontal_View: [Errno 2] No such file or directory: '/content/newTestDataset/dataset/Images/anonymous_003-008-562-01_1728904090342_Frontal_View.jpg'
Error copying files for anonymous_003-007-847-01_1730184498927_Frontal_View: [Errno 2] No such file or directory: '/content/newTestDataset/

In [ ]:
!zip -r /content/Split_Test_Data_Final /content/Split_Test_Data

  adding: content/Split_Test_Data/ (stored 0%)
  adding: content/Split_Test_Data/NonCavity_Data/ (stored 0%)
  adding: content/Split_Test_Data/NonCavity_Data/labels/ (stored 0%)
  adding: content/Split_Test_Data/NonCavity_Data/labels/anonymous_003-007-848-00_1730185308013_Frontal_View.txt (deflated 25%)
  adding: content/Split_Test_Data/NonCavity_Data/labels/anonymous_003-007-950-00_1730792603619_Frontal_View.txt (deflated 27%)
  adding: content/Split_Test_Data/NonCavity_Data/labels/anonymous_003-008-816-00_1729939059509_Frontal_View.txt (deflated 18%)
  adding: content/Split_Test_Data/NonCavity_Data/labels/anonymous_003-007-673-00_1729239709286_Frontal_View.txt (deflated 31%)
  adding: content/Split_Test_Data/NonCavity_Data/images/ (stored 0%)
  adding: content/Split_Test_Data/NonCavity_Data/images/anonymous_003-008-816-00_1729939059509_Frontal_View.jpg (deflated 1%)
  adding: content/Split_Test_Data/NonCavity_Data/images/anonymous_003-007-848-00_1730185308013_Frontal_View.jpg (deflat

In [ ]:
import os
SOURCE_LABEL_DIR = '/content/newDataset/dataset/Labels'
print(f"Total files in Labels folder: {len(os.listdir(SOURCE_LABEL_DIR))}")

Total files in Labels folder: 23


In [ ]:
import os
from collections import Counter

SOURCE_IMAGE_DIR = '/content/newTestDataset/dataset/Images'

# Get all filenames in the Images directory
all_files = os.listdir(SOURCE_IMAGE_DIR)

# Extract extensions and count them
extensions = [os.path.splitext(f)[1].lower() for f in all_files if os.path.isfile(os.path.join(SOURCE_IMAGE_DIR, f))]

extension_counts = Counter(extensions)

print("File Extension Counts in Images folder:")
for ext, count in extension_counts.items():
    print(f"  {ext}: {count}")

File Extension Counts in Images folder:
  .jpg: 38


In [ ]:
# The original code's image search logic (simplified)
image_path = os.path.join(SOURCE_IMAGE_DIR, base_name + '.jpg')

if not os.path.exists(image_path):
    image_path_png = os.path.join(SOURCE_IMAGE_DIR, base_name + '.png')
    if os.path.exists(image_path_png):
        image_path = image_path_png # Use PNG path
    # *** PROBLEM: If the image is found as PNG, we continue,
    # but the image_path variable is only updated if it was found as PNG.
    # If it was still NOT found (e.g., if it's a different extension or missing)
    # the image_path still points to the non-existent .jpg, leading to the error.

In [ ]:
import os
import shutil
import glob

# --- Configuration (Verified) ---
BASE_SOURCE_DIR = '/content/newTestDataset/dataset'
SOURCE_IMAGE_DIR = os.path.join(BASE_SOURCE_DIR, 'Images')
SOURCE_LABEL_DIR = os.path.join(BASE_SOURCE_DIR, 'Labels')
BASE_DEST_DIR = '/content/Split_Test_Data_Final'
CAVITY_CLASS_INDEX = '0'

# --- 1. Define and Create Destination Folders ---
DESTINATIONS = {
    'Cavity': {
        'images': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'labels')
    },
    'NonCavity': {
        'images': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'labels')
    }
}

print(f"Creating necessary directories under {BASE_DEST_DIR}...")
for category in DESTINATIONS.values():
    os.makedirs(category['images'], exist_ok=True)
    os.makedirs(category['labels'], exist_ok=True)
    print(f"Created: {category['images']}")

# --- 2. Process and Move Files (FIXED LOGIC) ---

label_files = glob.glob(os.path.join(SOURCE_LABEL_DIR, '*.txt'))
print(f"\nFound {len(label_files)} label files to process.")

cavity_count = 0
non_cavity_count = 0
skipped_count = 0

for label_path in label_files:
    base_name = os.path.splitext(os.path.basename(label_path))[0]

    # --- FIX START: Robustly determine the image path with naming flexibility ---
    image_path = None

    # 1. Check for a direct match (e.g., if label file has dashes, check for image with dashes)
    candidate_original = os.path.join(SOURCE_IMAGE_DIR, base_name + '.jpg')
    if os.path.exists(candidate_original):
        image_path = candidate_original

    # 2. Check for the alternate name (replace all dashes with underscores)
    if image_path is None:
        alternate_base_name = base_name.replace('-', '_')
        candidate_alternate = os.path.join(SOURCE_IMAGE_DIR, alternate_base_name + '.jpg')
        if os.path.exists(candidate_alternate):
            image_path = candidate_alternate

    # 3. Check for the reverse alternate name (replace all underscores with dashes)
    # This handles cases where the label file might use underscores and the image uses dashes
    if image_path is None:
        reverse_alternate_base_name = base_name.replace('_', '-')
        candidate_reverse = os.path.join(SOURCE_IMAGE_DIR, reverse_alternate_base_name + '.jpg')
        if os.path.exists(candidate_reverse):
            image_path = candidate_reverse

    # If image_path is still None, the image file is truly missing
    if image_path is None:
        print(f"Warning: Skipping {base_name}. Image (.jpg) not found with original or alternate naming convention.")
        skipped_count += 1
        continue # Skip to the next label file

    # --- FIX END: image_path is now confirmed to be a valid path ---

    is_cavity_data = True

    # 2. Read the label file content (Unchanged logic)
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip()
            if line:
                parts = line.split()
                if parts and parts[0] != CAVITY_CLASS_INDEX:
                    is_cavity_data = False
                    break

    except Exception as e:
        print(f"Error reading or processing {label_path}: {e}")
        is_cavity_data = False

    # 4. Move (Copy) the files

    if is_cavity_data:
        dest_set = DESTINATIONS['Cavity']
        cavity_count += 1
    else:
        dest_set = DESTINATIONS['NonCavity']
        non_cavity_count += 1

    try:
        shutil.copy(image_path, dest_set['images'])
        shutil.copy(label_path, dest_set['labels'])
    except Exception as e:
        # This error is now highly unlikely, but kept for robustness
        print(f"Severe Error during final copy for {base_name}: {e}")

# --- 3. Final Summary ---
print("\n--- Summary ---")
print(f"✅ Processing Complete.")
print(f"Files successfully classified and moved: {cavity_count + non_cavity_count}")
print(f"Moved **{cavity_count}** files to the Cavity folders.")
print(f"Moved **{non_count}** files to the NonCavity folders.")
print(f"Files skipped due to missing image: {skipped_count}")
print(f"Total label files processed: {len(label_files)}")

Creating necessary directories under /content/Split_Test_Data_Final...
Created: /content/Split_Test_Data_Final/Cavity_Data/images
Created: /content/Split_Test_Data_Final/NonCavity_Data/images

Found 38 label files to process.

--- Summary ---
✅ Processing Complete.
Files successfully classified and moved: 15
Moved **11** files to the Cavity folders.


NameError: name 'non_count' is not defined

In [ ]:
import os
import shutil
import glob

# --- Configuration (Verified) ---
BASE_SOURCE_DIR = '/content/newTestDataset/dataset'
SOURCE_IMAGE_DIR = os.path.join(BASE_SOURCE_DIR, 'Images')
SOURCE_LABEL_DIR = os.path.join(BASE_SOURCE_DIR, 'Labels')
BASE_DEST_DIR = '/content/Test_Data_Final'
CAVITY_CLASS_INDEX = '0'

# --- 1. Define and Create Destination Folders ---
DESTINATIONS = {
    'Cavity': {
        'images': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'labels')
    },
    'NonCavity': {
        'images': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'labels')
    }
}

print(f"Creating necessary directories under {BASE_DEST_DIR}...")
for category in DESTINATIONS.values():
    os.makedirs(category['images'], exist_ok=True)
    os.makedirs(category['labels'], exist_ok=True)
    print(f"Created: {category['images']}")

# --- 2. Process and Move Files (FIXED LOGIC) ---

label_files = glob.glob(os.path.join(SOURCE_LABEL_DIR, '*.txt'))
print(f"\nFound {len(label_files)} label files to process.")

cavity_count = 0
non_cavity_count = 0
skipped_count = 0

for label_path in label_files:
    base_name = os.path.splitext(os.path.basename(label_path))[0]

    # --- FIX START: Robustly determine the image path with naming flexibility ---
    image_path = None

    # 1. Check for a direct match (e.g., if label file has dashes, check for image with dashes)
    candidate_original = os.path.join(SOURCE_IMAGE_DIR, base_name + '.jpg')
    if os.path.exists(candidate_original):
        image_path = candidate_original

    # 2. Check for the alternate name (replace all dashes with underscores)
    if image_path is None:
        alternate_base_name = base_name.replace('-', '_')
        candidate_alternate = os.path.join(SOURCE_IMAGE_DIR, alternate_base_name + '.jpg')
        if os.path.exists(candidate_alternate):
            image_path = candidate_alternate

    # 3. Check for the reverse alternate name (replace all underscores with dashes)
    # This handles cases where the label file might use underscores and the image uses dashes
    if image_path is None:
        reverse_alternate_base_name = base_name.replace('_', '-')
        candidate_reverse = os.path.join(SOURCE_IMAGE_DIR, reverse_alternate_base_name + '.jpg')
        if os.path.exists(candidate_reverse):
            image_path = candidate_reverse

    # If image_path is still None, the image file is truly missing
    if image_path is None:
        print(f"Warning: Skipping {base_name}. Image (.jpg) not found with original or alternate naming convention.")
        skipped_count += 1
        continue # Skip to the next label file

    # --- FIX END: image_path is now confirmed to be a valid path ---

    is_cavity_data = True

    # 2. Read the label file content (Unchanged logic)
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip()
            if line:
                parts = line.split()
                if parts and parts[0] != CAVITY_CLASS_INDEX:
                    is_cavity_data = False
                    break

    except Exception as e:
        print(f"Error reading or processing {label_path}: {e}")
        is_cavity_data = False

    # 4. Move (Copy) the files

    if is_cavity_data:
        dest_set = DESTINATIONS['Cavity']
        cavity_count += 1
    else:
        dest_set = DESTINATIONS['NonCavity']
        non_cavity_count += 1

    try:
        shutil.copy(image_path, dest_set['images'])
        shutil.copy(label_path, dest_set['labels'])
    except Exception as e:
        # This error is now highly unlikely, but kept for robustness
        print(f"Severe Error during final copy for {base_name}: {e}")

# --- 3. Final Summary ---
print("\n--- Summary ---")
print(f"✅ Processing Complete.")
print(f"Files successfully classified and moved: {cavity_count + non_cavity_count}")
print(f"Moved **{cavity_count}** files to the Cavity folders.")
print(f"Moved **{non_count}** files to the NonCavity folders.")
print(f"Files skipped due to missing image: {skipped_count}")
print(f"Total label files processed: {len(label_files)}")

Creating necessary directories under /content/Test_Data_Final...
Created: /content/Test_Data_Final/Cavity_Data/images
Created: /content/Test_Data_Final/NonCavity_Data/images

Found 38 label files to process.

--- Summary ---
✅ Processing Complete.
Files successfully classified and moved: 15
Moved **11** files to the Cavity folders.


NameError: name 'non_count' is not defined

In [ ]:
import os
import shutil
import glob

# --- Configuration (Verified) ---
BASE_SOURCE_DIR = '/content/newTestDataset/dataset'
SOURCE_IMAGE_DIR = os.path.join(BASE_SOURCE_DIR, 'Images')
SOURCE_LABEL_DIR = os.path.join(BASE_SOURCE_DIR, 'Labels')
BASE_DEST_DIR = '/content/Split_Test_Data_Final'
CAVITY_CLASS_INDEX = '0'

# --- 1. Define and Create Destination Folders ---
DESTINATIONS = {
    'Cavity': {
        'images': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'Cavity_Data', 'labels')
    },
    'NonCavity': {
        'images': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'images'),
        'labels': os.path.join(BASE_DEST_DIR, 'NonCavity_Data', 'labels')
    }
}

print(f"Creating necessary directories under {BASE_DEST_DIR}...")
for category in DESTINATIONS.values():
    os.makedirs(category['images'], exist_ok=True)
    os.makedirs(category['labels'], exist_ok=True)
    print(f"Created: {category['images']}")

# --- 2. Process and Move Files ---

label_files = glob.glob(os.path.join(SOURCE_LABEL_DIR, '*.txt'))
print(f"\nFound {len(label_files)} label files to process.")

cavity_count = 0
non_cavity_count = 0  # <--- CORRECTLY INITIALIZED
skipped_count = 0

for label_path in label_files:
    base_name = os.path.splitext(os.path.basename(label_path))[0]

    # Robustly determine the image path with naming flexibility (dashes/underscores)
    image_path = None

    # 1. Check for a direct match (.jpg only, as confirmed by user)
    candidate_original = os.path.join(SOURCE_IMAGE_DIR, base_name + '.jpg')
    if os.path.exists(candidate_original):
        image_path = candidate_original

    # 2. Check for the alternate name (replace dashes with underscores)
    if image_path is None:
        alternate_base_name = base_name.replace('-', '_')
        candidate_alternate = os.path.join(SOURCE_IMAGE_DIR, alternate_base_name + '.jpg')
        if os.path.exists(candidate_alternate):
            image_path = candidate_alternate

    # 3. Check for the reverse alternate name (replace underscores with dashes)
    if image_path is None:
        reverse_alternate_base_name = base_name.replace('_', '-')
        candidate_reverse = os.path.join(SOURCE_IMAGE_DIR, reverse_alternate_base_name + '.jpg')
        if os.path.exists(candidate_reverse):
            image_path = candidate_reverse

    # If image_path is still None, the image file is truly missing
    if image_path is None:
        print(f"Warning: Skipping {base_name}. Image (.jpg) not found with original or alternate naming convention.")
        skipped_count += 1
        continue

    is_cavity_data = True

    # Read the label file content and classify
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip()
            if line:
                parts = line.split()
                if parts and parts[0] != CAVITY_CLASS_INDEX:
                    is_cavity_data = False
                    break

    except Exception as e:
        print(f"Error reading or processing {label_path}: {e}")
        is_cavity_data = False

    # Determine destination and copy files

    if is_cavity_data:
        dest_set = DESTINATIONS['Cavity']
        cavity_count += 1
    else:
        dest_set = DESTINATIONS['NonCavity']
        non_cavity_count += 1 # <--- CORRECT VARIABLE INCREMENTED

    try:
        shutil.copy(image_path, dest_set['images'])
        shutil.copy(label_path, dest_set['labels'])
    except Exception as e:
        print(f"Severe Error during final copy for {base_name}: {e}")

# --- 3. Final Summary (CORRECTED) ---
print("\n--- Summary ---")
print(f"✅ Processing Complete.")
print(f"Files successfully classified and moved: {cavity_count + non_cavity_count}")
print(f"Moved **{cavity_count}** files to the Cavity folders.")
print(f"Moved **{non_cavity_count}** files to the NonCavity folders.") # <--- CORRECTED LINE
print(f"Files skipped due to missing image: {skipped_count}")
print(f"Total label files processed: {len(label_files)}")

Creating necessary directories under /content/Split_Test_Data_Final...
Created: /content/Split_Test_Data_Final/Cavity_Data/images
Created: /content/Split_Test_Data_Final/NonCavity_Data/images

Found 38 label files to process.

--- Summary ---
✅ Processing Complete.
Files successfully classified and moved: 15
Moved **11** files to the Cavity folders.
Moved **4** files to the NonCavity folders.
Files skipped due to missing image: 23
Total label files processed: 38


In [ ]:
!unzip /content/dataset.zip -d /content/newTestDataset/

Archive:  /content/dataset.zip
   creating: /content/newTestDataset/dataset/Images/
   creating: /content/newTestDataset/dataset/Labels/
  inflating: /content/newTestDataset/dataset/Images/anonymous_003-007-1130-00_1732522239159_Frontal_View.jpg  
  inflating: /content/newTestDataset/dataset/Images/anonymous_003-007-1208-00_1732856242544_Frontal_View.jpg  
  inflating: /content/newTestDataset/dataset/Images/anonymous_003-007-1212-00_1732860958536_Frontal_View.jpg  
  inflating: /content/newTestDataset/dataset/Images/anonymous_003-007-1214-00_1732862917124_Frontal_View.jpg  
  inflating: /content/newTestDataset/dataset/Images/anonymous_003-007-1361-00_1733724284664_Frontal_View.jpg  
  inflating: /content/newTestDataset/dataset/Images/anonymous_003-007-1477-00_1734160734308_Frontal_View.jpg  
  inflating: /content/newTestDataset/dataset/Images/anonymous_003-007-607-00_1729059680698_Frontal_View.jpg  
  inflating: /content/newTestDataset/dataset/Images/anonymous_003-007-673-00_1729239709

In [ ]:
import os
import shutil
import zipfile
from tqdm import tqdm

# --- Configuration ---
SOURCE_DIR = "/content/newTestDataset/dataset"
OUTPUT_ROOT = "/content/Split_Test_Data_Final"
CAVITY_CLASS_INDEX = "0"  # Note: The class index in the label file is a string

IMAGES_SUBDIR = "Images"
LABELS_SUBDIR = "Labels"

# Define full paths
SOURCE_IMAGES = os.path.join(SOURCE_DIR, IMAGES_SUBDIR)
SOURCE_LABELS = os.path.join(SOURCE_DIR, LABELS_SUBDIR)

# --- Destination Folders ---
CAVITY_DIR = os.path.join(OUTPUT_ROOT, "Cavity_Data")
NON_CAVITY_DIR = os.path.join(OUTPUT_ROOT, "NonCavity_Data")

DESTINATIONS = {
    "Cavity": CAVITY_DIR,
    "NonCavity": NON_CAVITY_DIR
}

# --- Utility Functions ---

def create_output_directories():
    """Creates all necessary output directories."""
    print("Creating destination directories...")
    for key, root_dir in DESTINATIONS.items():
        os.makedirs(os.path.join(root_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(root_dir, "labels"), exist_ok=True)
        print(f"Created/Ensured: {root_dir}")

def determine_image_path(label_filename_base, source_images_dir):
    """
    Finds the corresponding image file by handling filename mismatches (dashes vs. underscores).

    Args:
        label_filename_base (str): Filename without extension (e.g., 'image-001')
        source_images_dir (str): Path to the source images folder

    Returns:
        str or None: The full path to the image file, or None if not found.
    """
    # 1. Standard check (e.g., label_filename_base is 'image_001', image is 'image_001.jpg')
    image_path_std = os.path.join(source_images_dir, f"{label_filename_base}.jpg")
    if os.path.exists(image_path_std):
        return image_path_std

    # 2. Mismatch check: Swap underscores and dashes in the base name
    #    (e.g., label is 'image-001.txt', but image is 'image_001.jpg')
    if '_' in label_filename_base:
        alt_base = label_filename_base.replace('_', '-')
    elif '-' in label_filename_base:
        alt_base = label_filename_base.replace('-', '_')
    else:
        alt_base = None

    if alt_base:
        image_path_alt = os.path.join(source_images_dir, f"{alt_base}.jpg")
        if os.path.exists(image_path_alt):
            return image_path_alt

    return None

def classify_and_migrate():
    """Performs the classification and file migration."""
    counts = {"Cavity": 0, "NonCavity": 0, "MissingImage": 0}

    label_files = [f for f in os.listdir(SOURCE_LABELS) if f.endswith(".txt")]

    print(f"\nProcessing {len(label_files)} label files...")

    for label_filename in tqdm(label_files, desc="Processing files"):
        label_file_path = os.path.join(SOURCE_LABELS, label_filename)
        base_name, _ = os.path.splitext(label_filename)

        # --- Classification ---
        try:
            is_cavity_only = True
            with open(label_file_path, 'r') as f:
                content = f.read().strip()

            if not content: # Empty label file
                # Treat an empty file as Non-Cavity (it violates the 'Cavity-only' rule)
                classification = "NonCavity"
            else:
                for line in content.split('\n'):
                    if line.strip() and not line.startswith(CAVITY_CLASS_INDEX):
                        is_cavity_only = False
                        break

                classification = "Cavity" if is_cavity_only else "NonCavity"

        except Exception as e:
            print(f"Error reading/classifying {label_filename}: {e}")
            continue

        # --- File Migration ---

        # 1. Find the corresponding image
        image_path = determine_image_path(base_name, SOURCE_IMAGES)

        if not image_path:
            counts["MissingImage"] += 1
            # print(f"Warning: Corresponding image not found for {label_filename}. Skipping.")
            continue

        # 2. Define destinations
        dest_root = DESTINATIONS[classification]
        dest_image_dir = os.path.join(dest_root, "images")
        dest_label_dir = os.path.join(dest_root, "labels")

        # Use the name of the found image for consistency in the destination
        final_image_filename = os.path.basename(image_path)
        final_label_filename = f"{os.path.splitext(final_image_filename)[0]}.txt"

        # 3. Copy files (using copy to keep the source dataset intact)
        try:
            # Copy Image
            shutil.copy2(image_path, os.path.join(dest_image_dir, final_image_filename))
            # Copy Label
            shutil.copy2(label_file_path, os.path.join(dest_label_dir, final_label_filename))

            counts[classification] += 1

        except Exception as e:
            print(f"Error copying files for {base_name}: {e}")

    return counts

def zip_output_folder(folder_to_zip):
    """Zips the output folder for easy download."""
    zip_filename = f"{os.path.basename(folder_to_zip)}.zip"

    print(f"\nZipping {folder_to_zip} to {zip_filename}...")

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through all the files in the output directory
        for root, dirs, files in os.walk(folder_to_zip):
            for file in files:
                full_path = os.path.join(root, file)
                # Create a relative path to keep the zip file structure clean
                rel_path = os.path.relpath(full_path, os.path.dirname(folder_to_zip))
                zipf.write(full_path, rel_path)

    print(f"Successfully created: **{zip_filename}**")
    return zip_filename

# --- Main Execution ---

if __name__ == "__main__":
    create_output_directories()

    final_counts = classify_and_migrate()

    print("\n" + "="*40)
    print("✨ Classification and Migration Summary ✨")
    print(f"Source Directory: **{SOURCE_DIR}**")
    print(f"Target Directory: **{OUTPUT_ROOT}**")
    print("---")
    print(f"**Cavity Data Pairs:** {final_counts['Cavity']}")
    print(f"**Non-Cavity Data Pairs:** {final_counts['NonCavity']}")
    print(f"Files Skipped (Image Missing): {final_counts['MissingImage']}")
    print("="*40)

    # Final Zipping step
    zip_path = zip_output_folder(OUTPUT_ROOT)

Creating destination directories...
Created/Ensured: /content/Split_Test_Data_Final/Cavity_Data
Created/Ensured: /content/Split_Test_Data_Final/NonCavity_Data

Processing 38 label files...


Processing files: 100%|██████████| 38/38 [00:00<00:00, 2165.92it/s]


✨ Classification and Migration Summary ✨
Source Directory: **/content/newTestDataset/dataset**
Target Directory: **/content/Split_Test_Data_Final**
---
**Cavity Data Pairs:** 9
**Non-Cavity Data Pairs:** 6
Files Skipped (Image Missing): 23

Zipping /content/Split_Test_Data_Final to Split_Test_Data_Final.zip...
Successfully created: **Split_Test_Data_Final.zip**


In [ ]:
!unzip /content/newTestDataset/test_Dataset.zip -d newTest

Archive:  /content/newTestDataset/test_Dataset.zip
   creating: newTest/test_Dataset/images/
   creating: newTest/test_Dataset/labels/
  inflating: newTest/test_Dataset/images/1000_jpg.rf.2a4ee9ac3855bad656f47d54ab898ae9.jpg  
  inflating: newTest/test_Dataset/images/1001_jpg.rf.1f05aceb3c2291ca71941bb71404e918.jpg  
  inflating: newTest/test_Dataset/images/1002_jpg.rf.cf7e006e4be03124d4c2b9706e01c996.jpg  
  inflating: newTest/test_Dataset/images/1009_jpg.rf.af373031d2b05184bde9c842b4b5b9d9.jpg  
  inflating: newTest/test_Dataset/images/1036_jpg.rf.bc23aeca16362facf9f7936c35f6e8b5.jpg  
  inflating: newTest/test_Dataset/images/1046_jpg.rf.1b3427a8e0f1be3587c36d83a9f9d91e.jpg  
  inflating: newTest/test_Dataset/images/1058_jpg.rf.b4121cfb02ced51b49f8718e0f96b283.jpg  
  inflating: newTest/test_Dataset/images/1062_jpg.rf.0c14ffe775754c373c59c63a58e573d5.jpg  
  inflating: newTest/test_Dataset/images/1069_jpg.rf.31ffc80e50ada30a20485fd4729b603c.jpg  
  inflating: newTest/test_Dataset/ima

In [ ]:
import os
import shutil
import zipfile
from tqdm import tqdm

# --- Configuration ---
SOURCE_DIR = "/content/newTest/test_Dataset"
OUTPUT_ROOT = "/content/Split_Test_Final"
CAVITY_CLASS_INDEX = "0"  # Note: The class index in the label file is a string

# 🐛 FIX: Change "Images" to "images" to match your dataset folder structure
IMAGES_SUBDIR = "images"
LABELS_SUBDIR = "Labels"

# Define full paths
SOURCE_IMAGES = os.path.join(SOURCE_DIR, IMAGES_SUBDIR)
SOURCE_LABELS = os.path.join(SOURCE_DIR, LABELS_SUBDIR)
# The corrected paths will now be:
# SOURCE_IMAGES = "/content/newTest/test_Dataset/images"
# SOURCE_LABELS = "/content/newTest/test_Dataset/Labels"

# --- Destination Folders ---
CAVITY_DIR = os.path.join(OUTPUT_ROOT, "Cavity_Data")
NON_CAVITY_DIR = os.path.join(OUTPUT_ROOT, "NonCavity_Data")

DESTINATIONS = {
    "Cavity": CAVITY_DIR,
    "NonCavity": NON_CAVITY_DIR
}

# --- Utility Functions ---

def create_output_directories():
    """Creates all necessary output directories."""
    print("Creating destination directories...")
    for key, root_dir in DESTINATIONS.items():
        os.makedirs(os.path.join(root_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(root_dir, "labels"), exist_ok=True)
        print(f"Created/Ensured: {root_dir}")

def determine_image_path(label_filename_base, source_images_dir):
    """
    Finds the corresponding image file by handling filename mismatches (dashes vs. underscores).

    Args:
        label_filename_base (str): Filename without extension (e.g., 'image-001')
        source_images_dir (str): Path to the source images folder

    Returns:
        str or None: The full path to the image file, or None if not found.
    """
    # 1. Standard check (e.g., label_filename_base is 'image_001', image is 'image_001.jpg')
    image_path_std = os.path.join(source_images_dir, f"{label_filename_base}.jpg")
    if os.path.exists(image_path_std):
        return image_path_std

    # 2. Mismatch check: Swap underscores and dashes in the base name
    #    (e.g., label is 'image-001.txt', but image is 'image_001.jpg')
    if '_' in label_filename_base:
        alt_base = label_filename_base.replace('_', '-')
    elif '-' in label_filename_base:
        alt_base = label_filename_base.replace('-', '_')
    else:
        alt_base = None

    if alt_base:
        image_path_alt = os.path.join(source_images_dir, f"{alt_base}.jpg")
        if os.path.exists(image_path_alt):
            return image_path_alt

    return None

def classify_and_migrate():
    """Performs the classification and file migration."""
    counts = {"Cavity": 0, "NonCavity": 0, "MissingImage": 0}

    label_files = [f for f in os.listdir(SOURCE_LABELS) if f.endswith(".txt")]

    print(f"\nProcessing {len(label_files)} label files...")

    for label_filename in tqdm(label_files, desc="Processing files"):
        label_file_path = os.path.join(SOURCE_LABELS, label_filename)
        base_name, _ = os.path.splitext(label_filename)

        # --- Classification ---
        try:
            is_cavity_only = True
            with open(label_file_path, 'r') as f:
                content = f.read().strip()

            if not content: # Empty label file
                # Treat an empty file as Non-Cavity (it violates the 'Cavity-only' rule)
                classification = "NonCavity"
            else:
                for line in content.split('\n'):
                    if line.strip() and not line.startswith(CAVITY_CLASS_INDEX):
                        is_cavity_only = False
                        break

                classification = "Cavity" if is_cavity_only else "NonCavity"

        except Exception as e:
            print(f"Error reading/classifying {label_filename}: {e}")
            continue

        # --- File Migration ---

        # 1. Find the corresponding image
        image_path = determine_image_path(base_name, SOURCE_IMAGES)

        if not image_path:
            counts["MissingImage"] += 1
            # print(f"Warning: Corresponding image not found for {label_filename}. Skipping.")
            continue

        # 2. Define destinations
        dest_root = DESTINATIONS[classification]
        dest_image_dir = os.path.join(dest_root, "images")
        dest_label_dir = os.path.join(dest_root, "labels")

        # Use the name of the found image for consistency in the destination
        final_image_filename = os.path.basename(image_path)
        final_label_filename = f"{os.path.splitext(final_image_filename)[0]}.txt"

        # 3. Copy files (using copy to keep the source dataset intact)
        try:
            # Copy Image
            shutil.copy2(image_path, os.path.join(dest_image_dir, final_image_filename))
            # Copy Label
            shutil.copy2(label_file_path, os.path.join(dest_label_dir, final_label_filename))

            counts[classification] += 1

        except Exception as e:
            print(f"Error copying files for {base_name}: {e}")

    return counts

def zip_output_folder(folder_to_zip):
    """Zips the output folder for easy download."""
    zip_filename = f"{os.path.basename(folder_to_zip)}.zip"

    print(f"\nZipping {folder_to_zip} to {zip_filename}...")

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through all the files in the output directory
        for root, dirs, files in os.walk(folder_to_zip):
            for file in files:
                full_path = os.path.join(root, file)
                # Create a relative path to keep the zip file structure clean
                rel_path = os.path.relpath(full_path, os.path.dirname(folder_to_zip))
                zipf.write(full_path, rel_path)

    print(f"Successfully created: **{zip_filename}**")
    return zip_filename

# --- Main Execution ---

if __name__ == "__main__":
    create_output_directories()

    final_counts = classify_and_migrate()

    print("\n" + "="*40)
    print("✨ Classification and Migration Summary ✨")
    print(f"Source Directory: **{SOURCE_DIR}**")
    print(f"Target Directory: **{OUTPUT_ROOT}**")
    print("---")
    print(f"**Cavity Data Pairs:** {final_counts['Cavity']}")
    print(f"**Non-Cavity Data Pairs:** {final_counts['NonCavity']}")
    print(f"Files Skipped (Image Missing): {final_counts['MissingImage']}")
    print("="*40)

    # Final Zipping step
    zip_path = zip_output_folder(OUTPUT_ROOT)

Creating destination directories...
Created/Ensured: /content/Split_Test_Final/Cavity_Data
Created/Ensured: /content/Split_Test_Final/NonCavity_Data

Processing 253 label files...


Processing files: 100%|██████████| 253/253 [00:00<00:00, 872.19it/s]



✨ Classification and Migration Summary ✨
Source Directory: **/content/newTest/test_Dataset**
Target Directory: **/content/Split_Test_Final**
---
**Cavity Data Pairs:** 0
**Non-Cavity Data Pairs:** 253
Files Skipped (Image Missing): 0

Zipping /content/Split_Test_Final to Split_Test_Final.zip...
Successfully created: **Split_Test_Final.zip**


Code for the **Testing Images** and seperating them into two categories of Cavity and Non Cavity

In [ ]:
!unzip /content/test_Dataset.zip -d test_dataset_split


Archive:  /content/test_Dataset.zip
   creating: test_dataset_split/test_Dataset/images/
   creating: test_dataset_split/test_Dataset/labels/
  inflating: test_dataset_split/test_Dataset/images/-1-_jpg.rf.058c0f58313fde09037befc8a41e2e08.jpg  
  inflating: test_dataset_split/test_Dataset/images/-132-_jpg.rf.d50d3a76350f11dbd4989aff05e78083.jpg  
  inflating: test_dataset_split/test_Dataset/images/-140-_jpg.rf.b21e5a4fb0f4dbb35bb0d52ff70b9923.jpg  
  inflating: test_dataset_split/test_Dataset/images/-191-_jpg.rf.66af147c00eed9d2accae3b804254f3f.jpg  
  inflating: test_dataset_split/test_Dataset/images/-585-_jpg.rf.6687c99654c76d369cedef0dbb5f82b5.jpg  
  inflating: test_dataset_split/test_Dataset/images/00003_png_jpg.rf.3d930d122e118d917786f0b2cb1c3787.jpg  
  inflating: test_dataset_split/test_Dataset/images/00044_png_jpg.rf.518f71c5905d01e1c6619e5d079bd867.jpg  
  inflating: test_dataset_split/test_Dataset/images/00072_png_jpg.rf.a28883178ec60d4a817c5f56224541d0.jpg  
  inflating: tes

In [ ]:
import os
import shutil
import zipfile
from tqdm import tqdm

# --- Configuration ---
SOURCE_DIR = "/content/test_dataset_split/test_Dataset"
OUTPUT_ROOT = "/content/TestSplitDataset"
CAVITY_CLASS_INDEX = "0"  # Target class index for Cavity

# Corrected subdirectories based on your structure
IMAGES_SUBDIR = "images"
LABELS_SUBDIR = "labels"

# Define full paths
SOURCE_IMAGES = os.path.join(SOURCE_DIR, IMAGES_SUBDIR)
SOURCE_LABELS = os.path.join(SOURCE_DIR, LABELS_SUBDIR)

# --- Destination Folders ---
CAVITY_DIR = os.path.join(OUTPUT_ROOT, "Cavity_Data")
NON_CAVITY_DIR = os.path.join(OUTPUT_ROOT, "NonCavity_Data")

DESTINATIONS = {
    "Cavity": CAVITY_DIR,
    "NonCavity": NON_CAVITY_DIR
}

# --- Utility Functions ---

def create_output_directories():
    """Creates all necessary output directories."""
    print("Creating destination directories...")
    for key, root_dir in DESTINATIONS.items():
        os.makedirs(os.path.join(root_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(root_dir, "labels"), exist_ok=True)
        print(f"Created/Ensured: {root_dir}")

def determine_image_path(label_filename_base, source_images_dir):
    """
    Finds the corresponding image file by handling filename mismatches (dashes vs. underscores).
    """
    # 1. Standard check
    image_path_std = os.path.join(source_images_dir, f"{label_filename_base}.jpg")
    if os.path.exists(image_path_std):
        return image_path_std

    # 2. Mismatch check: Swap underscores and dashes
    if '_' in label_filename_base:
        alt_base = label_filename_base.replace('_', '-')
    elif '-' in label_filename_base:
        alt_base = label_filename_base.replace('-', '_')
    else:
        alt_base = None

    if alt_base:
        image_path_alt = os.path.join(source_images_dir, f"{alt_base}.jpg")
        if os.path.exists(image_path_alt):
            return image_path_alt

    return None

def classify_and_migrate():
    """Performs the classification and file migration using the REVISED rule."""
    counts = {"Cavity": 0, "NonCavity": 0, "MissingImage": 0}

    # Ensure source label directory exists before listing files
    if not os.path.isdir(SOURCE_LABELS):
        print(f"Error: Label directory not found at {SOURCE_LABELS}")
        return counts

    label_files = [f for f in os.listdir(SOURCE_LABELS) if f.endswith(".txt")]

    print(f"\nProcessing {len(label_files)} label files...")

    for label_filename in tqdm(label_files, desc="Processing files"):
        label_file_path = os.path.join(SOURCE_LABELS, label_filename)
        base_name, _ = os.path.splitext(label_filename)

        # --- Classification (REVISED LOGIC) ---
        try:
            # Check for the presence of the CAVITY_CLASS_INDEX ("0")
            contains_cavity = False

            with open(label_file_path, 'r') as f:
                content = f.read().strip()

            if content: # Only check if the file is not empty
                for line in content.split('\n'):
                    # Rule: Cavity if AT LEAST ONE line starts with "0"
                    if line.strip() and line.startswith(CAVITY_CLASS_INDEX):
                        contains_cavity = True
                        break

                # New Rule: Cavity if it contains class 0, NonCavity otherwise (including empty files)
                classification = "Cavity" if contains_cavity else "NonCavity"
            else:
                # Empty files contain no class 0, so they are NonCavity
                classification = "NonCavity"

        except Exception as e:
            print(f"Error reading/classifying {label_filename}: {e}")
            continue

        # --- File Migration ---

        # 1. Find the corresponding image
        image_path = determine_image_path(base_name, SOURCE_IMAGES)

        if not image_path:
            counts["MissingImage"] += 1
            continue

        # 2. Define destinations
        dest_root = DESTINATIONS[classification]
        dest_image_dir = os.path.join(dest_root, "images")
        dest_label_dir = os.path.join(dest_root, "labels")

        # Use the name of the found image for consistency in the destination
        final_image_filename = os.path.basename(image_path)
        final_label_filename = f"{os.path.splitext(final_image_filename)[0]}.txt"

        # 3. Copy files (using copy to keep the source dataset intact)
        try:
            shutil.copy2(image_path, os.path.join(dest_image_dir, final_image_filename))
            shutil.copy2(label_file_path, os.path.join(dest_label_dir, final_label_filename))

            counts[classification] += 1

        except Exception as e:
            print(f"Error copying files for {base_name}: {e}")

    return counts

def zip_output_folder(folder_to_zip):
    """Zips the output folder for easy download."""
    zip_filename = f"{os.path.basename(folder_to_zip)}.zip"

    print(f"\nZipping {folder_to_zip} to {zip_filename}...")

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through all the files in the output directory
        for root, dirs, files in os.walk(folder_to_zip):
            for file in files:
                full_path = os.path.join(root, file)
                # Create a relative path to keep the zip file structure clean
                rel_path = os.path.relpath(full_path, os.path.dirname(folder_to_zip))
                zipf.write(full_path, rel_path)

    print(f"Successfully created: **{zip_filename}**")
    return zip_filename

# --- Main Execution ---

if __name__ == "__main__":
    create_output_directories()

    final_counts = classify_and_migrate()

    print("\n" + "="*40)
    print("✨ Classification and Migration Summary (Revised Rule) ✨")
    print(f"Source Directory: **{SOURCE_DIR}**")
    print(f"Target Directory: **{OUTPUT_ROOT}**")
    print("---")
    print(f"**Cavity Data Pairs (Contains Class 0):** {final_counts['Cavity']}")
    print(f"**Non-Cavity Data Pairs (No Class 0):** {final_counts['NonCavity']}")
    print(f"Files Skipped (Image Missing): {final_counts['MissingImage']}")
    print("="*40)

    # Final Zipping step
    zip_path = zip_output_folder(OUTPUT_ROOT)

Creating destination directories...
Created/Ensured: /content/TestSplitDataset/Cavity_Data
Created/Ensured: /content/TestSplitDataset/NonCavity_Data

Processing 940 label files...


Processing files: 100%|██████████| 940/940 [00:00<00:00, 3912.90it/s]



✨ Classification and Migration Summary (Revised Rule) ✨
Source Directory: **/content/test_dataset_split/test_Dataset**
Target Directory: **/content/TestSplitDataset**
---
**Cavity Data Pairs (Contains Class 0):** 752
**Non-Cavity Data Pairs (No Class 0):** 188
Files Skipped (Image Missing): 0

Zipping /content/TestSplitDataset to TestSplitDataset.zip...
Successfully created: **TestSplitDataset.zip**


Code for the **Validation Images** and seperating them into two categories of Cavity and Non Cavity

In [ ]:
!unzip /content/valid_dataset.zip -d valid_dataset_split

Archive:  /content/valid_dataset.zip
   creating: valid_dataset_split/valid_dataset/images/
   creating: valid_dataset_split/valid_dataset/labels/
  inflating: valid_dataset_split/valid_dataset/images/-129-_jpg.rf.2400fa08d2e0beb9f6dbda37e15b8b97.jpg  
  inflating: valid_dataset_split/valid_dataset/images/-130-_jpg.rf.b5176d20873381918404842b7e708e3d.jpg  
  inflating: valid_dataset_split/valid_dataset/images/-159-_jpg.rf.aed6dba8f324d6e58810371d6c433a61.jpg  
  inflating: valid_dataset_split/valid_dataset/images/-192-_jpg.rf.cfe06bfce11bcc329fb71666dbb46d3c.jpg  
  inflating: valid_dataset_split/valid_dataset/images/-221-_jpg.rf.c66e20fcd1f3b7222ab7c43add7dc9ed.jpg  
  inflating: valid_dataset_split/valid_dataset/images/-48-_jpg.rf.e83b4b8e83b7cf33e0a41cb42fe97cd7.jpg  
  inflating: valid_dataset_split/valid_dataset/images/003_00036_04_a091_jpg.rf.17558bad72bf8e1044fed7c640742681.jpg  
  inflating: valid_dataset_split/valid_dataset/images/003_00036_04_a091_jpg.rf.4d1c05ee9b98d066e3eaf

In [ ]:
import os
import shutil
import zipfile
from tqdm import tqdm

# --- Configuration ---
SOURCE_DIR = "/content/valid_dataset_split/valid_dataset"
OUTPUT_ROOT = "/content/ValidSplitDataset"
CAVITY_CLASS_INDEX = "0"  # Target class index for Cavity

# Corrected subdirectories based on your structure
IMAGES_SUBDIR = "images"
LABELS_SUBDIR = "labels"

# Define full paths
SOURCE_IMAGES = os.path.join(SOURCE_DIR, IMAGES_SUBDIR)
SOURCE_LABELS = os.path.join(SOURCE_DIR, LABELS_SUBDIR)

# --- Destination Folders ---
CAVITY_DIR = os.path.join(OUTPUT_ROOT, "Cavity_Data")
NON_CAVITY_DIR = os.path.join(OUTPUT_ROOT, "NonCavity_Data")

DESTINATIONS = {
    "Cavity": CAVITY_DIR,
    "NonCavity": NON_CAVITY_DIR
}

# --- Utility Functions ---

def create_output_directories():
    """Creates all necessary output directories."""
    print("Creating destination directories...")
    for key, root_dir in DESTINATIONS.items():
        os.makedirs(os.path.join(root_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(root_dir, "labels"), exist_ok=True)
        print(f"Created/Ensured: {root_dir}")

def determine_image_path(label_filename_base, source_images_dir):
    """
    Finds the corresponding image file by handling filename mismatches (dashes vs. underscores).
    """
    # 1. Standard check
    image_path_std = os.path.join(source_images_dir, f"{label_filename_base}.jpg")
    if os.path.exists(image_path_std):
        return image_path_std

    # 2. Mismatch check: Swap underscores and dashes
    if '_' in label_filename_base:
        alt_base = label_filename_base.replace('_', '-')
    elif '-' in label_filename_base:
        alt_base = label_filename_base.replace('-', '_')
    else:
        alt_base = None

    if alt_base:
        image_path_alt = os.path.join(source_images_dir, f"{alt_base}.jpg")
        if os.path.exists(image_path_alt):
            return image_path_alt

    return None

def classify_and_migrate():
    """Performs the classification and file migration using the REVISED rule."""
    counts = {"Cavity": 0, "NonCavity": 0, "MissingImage": 0}

    # Ensure source label directory exists before listing files
    if not os.path.isdir(SOURCE_LABELS):
        print(f"Error: Label directory not found at {SOURCE_LABELS}")
        return counts

    label_files = [f for f in os.listdir(SOURCE_LABELS) if f.endswith(".txt")]

    print(f"\nProcessing {len(label_files)} label files...")

    for label_filename in tqdm(label_files, desc="Processing files"):
        label_file_path = os.path.join(SOURCE_LABELS, label_filename)
        base_name, _ = os.path.splitext(label_filename)

        # --- Classification (REVISED LOGIC) ---
        try:
            # Check for the presence of the CAVITY_CLASS_INDEX ("0")
            contains_cavity = False

            with open(label_file_path, 'r') as f:
                content = f.read().strip()

            if content: # Only check if the file is not empty
                for line in content.split('\n'):
                    # Rule: Cavity if AT LEAST ONE line starts with "0"
                    if line.strip() and line.startswith(CAVITY_CLASS_INDEX):
                        contains_cavity = True
                        break

                # New Rule: Cavity if it contains class 0, NonCavity otherwise (including empty files)
                classification = "Cavity" if contains_cavity else "NonCavity"
            else:
                # Empty files contain no class 0, so they are NonCavity
                classification = "NonCavity"

        except Exception as e:
            print(f"Error reading/classifying {label_filename}: {e}")
            continue

        # --- File Migration ---

        # 1. Find the corresponding image
        image_path = determine_image_path(base_name, SOURCE_IMAGES)

        if not image_path:
            counts["MissingImage"] += 1
            continue

        # 2. Define destinations
        dest_root = DESTINATIONS[classification]
        dest_image_dir = os.path.join(dest_root, "images")
        dest_label_dir = os.path.join(dest_root, "labels")

        # Use the name of the found image for consistency in the destination
        final_image_filename = os.path.basename(image_path)
        final_label_filename = f"{os.path.splitext(final_image_filename)[0]}.txt"

        # 3. Copy files (using copy to keep the source dataset intact)
        try:
            shutil.copy2(image_path, os.path.join(dest_image_dir, final_image_filename))
            shutil.copy2(label_file_path, os.path.join(dest_label_dir, final_label_filename))

            counts[classification] += 1

        except Exception as e:
            print(f"Error copying files for {base_name}: {e}")

    return counts

def zip_output_folder(folder_to_zip):
    """Zips the output folder for easy download."""
    zip_filename = f"{os.path.basename(folder_to_zip)}.zip"

    print(f"\nZipping {folder_to_zip} to {zip_filename}...")

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through all the files in the output directory
        for root, dirs, files in os.walk(folder_to_zip):
            for file in files:
                full_path = os.path.join(root, file)
                # Create a relative path to keep the zip file structure clean
                rel_path = os.path.relpath(full_path, os.path.dirname(folder_to_zip))
                zipf.write(full_path, rel_path)

    print(f"Successfully created: **{zip_filename}**")
    return zip_filename

# --- Main Execution ---

if __name__ == "__main__":
    create_output_directories()

    final_counts = classify_and_migrate()

    print("\n" + "="*40)
    print("✨ Classification and Migration Summary (Revised Rule) ✨")
    print(f"Source Directory: **{SOURCE_DIR}**")
    print(f"Target Directory: **{OUTPUT_ROOT}**")
    print("---")
    print(f"**Cavity Data Pairs (Contains Class 0):** {final_counts['Cavity']}")
    print(f"**Non-Cavity Data Pairs (No Class 0):** {final_counts['NonCavity']}")
    print(f"Files Skipped (Image Missing): {final_counts['MissingImage']}")
    print("="*40)

    # Final Zipping step
    zip_path = zip_output_folder(OUTPUT_ROOT)

Creating destination directories...
Created/Ensured: /content/ValidSplitDataset/Cavity_Data
Created/Ensured: /content/ValidSplitDataset/NonCavity_Data

Processing 1270 label files...


Processing files: 100%|██████████| 1270/1270 [00:00<00:00, 1365.09it/s]



✨ Classification and Migration Summary (Revised Rule) ✨
Source Directory: **/content/valid_dataset_split/valid_dataset**
Target Directory: **/content/ValidSplitDataset**
---
**Cavity Data Pairs (Contains Class 0):** 1009
**Non-Cavity Data Pairs (No Class 0):** 261
Files Skipped (Image Missing): 0

Zipping /content/ValidSplitDataset to ValidSplitDataset.zip...
Successfully created: **ValidSplitDataset.zip**


Code for the **Training Images** and seperating them into two categories of Cavity and Non Cavity

In [ ]:
!unzip /content/drive/MyDrive/trainDataset.zip -d train_dataset_split

Streaming output truncated to the last 5000 lines.
  inflating: train_dataset_split/trainDataset/labels/gingivitis-2123-_jpg.rf.4aa53640247b58678a8bb3a2de43ef1f.txt  
  inflating: train_dataset_split/trainDataset/labels/gingivitis-2125-_jpg.rf.b47af10a7fe965bc698e6a4ab8763d8d.txt  
  inflating: train_dataset_split/trainDataset/labels/gingivitis-2128-_jpg.rf.4e502c81843e763dcc4561db8e8b4afe.txt  
  inflating: train_dataset_split/trainDataset/labels/gingivitis-213-_jpg.rf.8458733b9da7b0c4858cea0cc1ca5e9e.txt  
  inflating: train_dataset_split/trainDataset/labels/gingivitis-2130-_jpg.rf.0126a6ce2c404beb3b7dd4aa6d2e5295.txt  
  inflating: train_dataset_split/trainDataset/labels/gingivitis-2131-_jpg.rf.8b06f829c800eb2efcc82cea42fdb5f2.txt  
  inflating: train_dataset_split/trainDataset/labels/gingivitis-2133-_jpg.rf.f1978e914a7aa7a3861601ceeecf7d93.txt  
  inflating: train_dataset_split/trainDataset/labels/gingivitis-2136-_jpg.rf.6cd69659425fe556dd5ab05783ada2d8.txt  
  inflating: train_dat

In [ ]:
import os
import shutil
import zipfile
from tqdm import tqdm

# --- Configuration ---
SOURCE_DIR = "/content/valid_dataset_split/valid_dataset"
OUTPUT_ROOT = "/content/ValidSplitDataset"
CAVITY_CLASS_INDEX = "0"  # Target class index for Cavity

# Corrected subdirectories based on your structure
IMAGES_SUBDIR = "images"
LABELS_SUBDIR = "labels"

# Define full paths
SOURCE_IMAGES = os.path.join(SOURCE_DIR, IMAGES_SUBDIR)
SOURCE_LABELS = os.path.join(SOURCE_DIR, LABELS_SUBDIR)

# --- Destination Folders ---
CAVITY_DIR = os.path.join(OUTPUT_ROOT, "Cavity_Data")
NON_CAVITY_DIR = os.path.join(OUTPUT_ROOT, "NonCavity_Data")

DESTINATIONS = {
    "Cavity": CAVITY_DIR,
    "NonCavity": NON_CAVITY_DIR
}

# --- Utility Functions ---

def create_output_directories():
    """Creates all necessary output directories."""
    print("Creating destination directories...")
    for key, root_dir in DESTINATIONS.items():
        os.makedirs(os.path.join(root_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(root_dir, "labels"), exist_ok=True)
        print(f"Created/Ensured: {root_dir}")

def determine_image_path(label_filename_base, source_images_dir):
    """
    Finds the corresponding image file by handling filename mismatches (dashes vs. underscores).
    """
    # 1. Standard check
    image_path_std = os.path.join(source_images_dir, f"{label_filename_base}.jpg")
    if os.path.exists(image_path_std):
        return image_path_std

    # 2. Mismatch check: Swap underscores and dashes
    if '_' in label_filename_base:
        alt_base = label_filename_base.replace('_', '-')
    elif '-' in label_filename_base:
        alt_base = label_filename_base.replace('-', '_')
    else:
        alt_base = None

    if alt_base:
        image_path_alt = os.path.join(source_images_dir, f"{alt_base}.jpg")
        if os.path.exists(image_path_alt):
            return image_path_alt

    return None

def classify_and_migrate():
    """Performs the classification and file migration using the REVISED rule."""
    counts = {"Cavity": 0, "NonCavity": 0, "MissingImage": 0}

    # Ensure source label directory exists before listing files
    if not os.path.isdir(SOURCE_LABELS):
        print(f"Error: Label directory not found at {SOURCE_LABELS}")
        return counts

    label_files = [f for f in os.listdir(SOURCE_LABELS) if f.endswith(".txt")]

    print(f"\nProcessing {len(label_files)} label files...")

    for label_filename in tqdm(label_files, desc="Processing files"):
        label_file_path = os.path.join(SOURCE_LABELS, label_filename)
        base_name, _ = os.path.splitext(label_filename)

        # --- Classification (REVISED LOGIC) ---
        try:
            # Check for the presence of the CAVITY_CLASS_INDEX ("0")
            contains_cavity = False

            with open(label_file_path, 'r') as f:
                content = f.read().strip()

            if content: # Only check if the file is not empty
                for line in content.split('\n'):
                    # Rule: Cavity if AT LEAST ONE line starts with "0"
                    if line.strip() and line.startswith(CAVITY_CLASS_INDEX):
                        contains_cavity = True
                        break

                # New Rule: Cavity if it contains class 0, NonCavity otherwise (including empty files)
                classification = "Cavity" if contains_cavity else "NonCavity"
            else:
                # Empty files contain no class 0, so they are NonCavity
                classification = "NonCavity"

        except Exception as e:
            print(f"Error reading/classifying {label_filename}: {e}")
            continue

        # --- File Migration ---

        # 1. Find the corresponding image
        image_path = determine_image_path(base_name, SOURCE_IMAGES)

        if not image_path:
            counts["MissingImage"] += 1
            continue

        # 2. Define destinations
        dest_root = DESTINATIONS[classification]
        dest_image_dir = os.path.join(dest_root, "images")
        dest_label_dir = os.path.join(dest_root, "labels")

        # Use the name of the found image for consistency in the destination
        final_image_filename = os.path.basename(image_path)
        final_label_filename = f"{os.path.splitext(final_image_filename)[0]}.txt"

        # 3. Copy files (using copy to keep the source dataset intact)
        try:
            shutil.copy2(image_path, os.path.join(dest_image_dir, final_image_filename))
            shutil.copy2(label_file_path, os.path.join(dest_label_dir, final_label_filename))

            counts[classification] += 1

        except Exception as e:
            print(f"Error copying files for {base_name}: {e}")

    return counts

def zip_output_folder(folder_to_zip):
    """Zips the output folder for easy download."""
    zip_filename = f"{os.path.basename(folder_to_zip)}.zip"

    print(f"\nZipping {folder_to_zip} to {zip_filename}...")

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through all the files in the output directory
        for root, dirs, files in os.walk(folder_to_zip):
            for file in files:
                full_path = os.path.join(root, file)
                # Create a relative path to keep the zip file structure clean
                rel_path = os.path.relpath(full_path, os.path.dirname(folder_to_zip))
                zipf.write(full_path, rel_path)

    print(f"Successfully created: **{zip_filename}**")
    return zip_filename

# --- Main Execution ---

if __name__ == "__main__":
    create_output_directories()

    final_counts = classify_and_migrate()

    print("\n" + "="*40)
    print("✨ Classification and Migration Summary (Revised Rule) ✨")
    print(f"Source Directory: **{SOURCE_DIR}**")
    print(f"Target Directory: **{OUTPUT_ROOT}**")
    print("---")
    print(f"**Cavity Data Pairs (Contains Class 0):** {final_counts['Cavity']}")
    print(f"**Non-Cavity Data Pairs (No Class 0):** {final_counts['NonCavity']}")
    print(f"Files Skipped (Image Missing): {final_counts['MissingImage']}")
    print("="*40)

    # Final Zipping step
    zip_path = zip_output_folder(OUTPUT_ROOT)

Streaming output truncated to the last 5000 lines.
  inflating: ValidSpDataset/ValidSplitDataset/Cavity_Data/labels/2471_jpg.rf.14f37051c65ce77143a4843fae2dd2b9.txt  
  inflating: ValidSpDataset/ValidSplitDataset/Cavity_Data/labels/2333_jpg.rf.56ed8b31766e3f596bd1f68478d764b2.txt  
  inflating: ValidSpDataset/ValidSplitDataset/Cavity_Data/labels/547_jpg.rf.11ac6948e941cf3276dc0bae9fd3e276.txt  
  inflating: ValidSpDataset/ValidSplitDataset/Cavity_Data/labels/2937_jpg.rf.788150176e1b7eb3ed8b7ce97fc9e9dd.txt  
  inflating: ValidSpDataset/ValidSplitDataset/Cavity_Data/labels/491_jpg.rf.cab647d459a0a0e8fd79eafb9827d0e7.txt  
  inflating: ValidSpDataset/ValidSplitDataset/Cavity_Data/labels/caries_0_4328_jpeg_jpg.rf.2f760af2bcd98f9e50597e632b5e9f3e.txt  
  inflating: ValidSpDataset/ValidSplitDataset/Cavity_Data/labels/880_jpg.rf.af4d519890c4ea50eb56da30c2113857.txt  
  inflating: ValidSpDataset/ValidSplitDataset/Cavity_Data/labels/186_jpg.rf.6c24d28adc648e2c9c4b1a6ae7e36251.txt  
  inflating